- NIM  : 2309106119
- Nama : Muhammad Akhyat Tariq Razan

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report

## 1. Load Dataset
- Membaca dan memproses dataset


In [4]:
df = pd.read_csv('online_gaming_behavior_dataset.csv')

# Tentukan kolom target
target_col = 'EngagementLevel'

# Pisahkan fitur dan target
X = df.drop(columns=[target_col])
y = df[target_col]

## 2. Encode kolom kategorikal
- Menentukan kolom target: `EngagementLevel`
- Melakukan One-Hot Encoding untuk fitur kategorikal



In [5]:
# Pilih kolom kategorikal
categorical_cols = X.select_dtypes(include=['object']).columns

# Ubah kategori menjadi numerik (One-hot encoding)
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

## 3. Normalisasi fitur numerik
- Melakukan normalisasi data dengan `StandardScaler`


In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 4. Split Data
- 80% data untuk training dan 20% untuk validation  
- Menggunakan `train_test_split` dengan `stratify` agar kelas seimbang

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)

## 5. Build Model (Sequential)
- Input layer sesuai jumlah fitur  
- Hidden layers: 128 neuron dan 64 neuron (aktivasi ReLU)  
- Dropout (0.3) untuk mencegah overfitting  
- Output layer: `Softmax` sesuai jumlah kelas

In [8]:
from keras import Input, Sequential
from keras.layers import Dense, Dropout

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,883 (42.51 KB)

 Trainable params: 10,883 (42.51 KB)

 Non-trainable params: 0 (0.00 B)

## 6. Train Model
- Optimizer: `Adam`
- Loss function: `Categorical Crossentropy`
- Epoch: 50, Batch size: 32

In [9]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.6842 - loss: 0.7441 - val_accuracy: 0.8579 - val_loss: 0.4689
Epoch 2/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8305 - loss: 0.5092 - val_accuracy: 0.8826 - val_loss: 0.4167
Epoch 3/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8529 - loss: 0.4605 - val_accuracy: 0.8947 - val_loss: 0.3973
Epoch 4/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8647 - loss: 0.4353 - val_accuracy: 0.8968 - val_loss: 0.3878
Epoch 5/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8719 - loss: 0.4302 - val_accuracy: 0.9018 - val_loss: 0.3801
Epoch 6/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8799 - loss: 0.4109 - val_accuracy: 0.8992 - val_loss: 0.3773
Epoch 7/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8796 - loss: 0.4152 - val_accuracy: 0.9026 - val_loss: 0.3725
Epoch 8/50
1001/1001 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8852 - loss: 0.3975 - 

## 7. Evaluasi
- Menghitung akurasi training dan validation  
- Menampilkan classification report (precision, recall, F1-score)

In [10]:
train_acc = model.evaluate(X_train, y_train, verbose=0)[1]
val_acc = model.evaluate(X_val, y_val, verbose=0)[1]

print(f"\nAkurasi Training: {train_acc*100:.2f}%")
print(f"Akurasi Validation: {val_acc*100:.2f}%")


Akurasi Training: 92.03%
Akurasi Validation: 91.38%


## 8. Laporan Klasifikasi
### Evaluasi Model

Setelah model selesai dilatih, dilakukan proses evaluasi untuk menilai kemampuan model dalam melakukan **klasifikasi** terhadap data validasi.

#### Langkah-langkah Evaluasi
1. **Prediksi Kelas**
   - Model menghasilkan probabilitas untuk setiap kelas melalui `model.predict(X_val)`.
   - Fungsi `np.argmax(..., axis=1)` digunakan untuk menentukan kelas dengan probabilitas tertinggi sebagai hasil prediksi (`y_pred`).

2. **Menentukan Label Sebenarnya**
   - Data label validasi (`y_val`) sebelumnya berbentuk one-hot encoding.  
   - Dengan `np.argmax(y_val, axis=1)`, label tersebut dikembalikan ke bentuk numerik (`y_true`) agar dapat dibandingkan dengan hasil prediksi.

3. **Menampilkan Laporan Klasifikasi**
   - Fungsi `classification_report(y_true, y_pred, target_names=le.classes_)` menampilkan metrik performa model berupa:
     - **Precision** → tingkat ketepatan model dalam memprediksi tiap kelas.  
     - **Recall** → kemampuan model mendeteksi data sebenarnya dari tiap kelas.  
     - **F1-Score** → rata-rata harmonis antara precision dan recall.  
     - **Support** → jumlah data pada tiap kelas.

###  Kesimpulan
Laporan klasifikasi ini memberikan gambaran menyeluruh tentang performa model dalam mengenali kategori pada kolom target **`EngagementLevel`**, sehingga dapat diketahui seberapa baik model melakukan prediksi terhadap perilaku pemain game online.

In [11]:
import numpy as np
y_pred = np.argmax(model.predict(X_val), axis=1)
y_true = np.argmax(y_val, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Classification Report:
              precision    recall  f1-score   support

        High       0.92      0.88      0.90      2067
         Low       0.91      0.88      0.90      2065
      Medium       0.91      0.95      0.93      3875

    accuracy                           0.91      8007
   macro avg       0.92      0.90      0.91      8007
weighted avg       0.91      0.91      0.91      8007

